In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_addons as tfa
from numpy.random import default_rng
rng = default_rng()


In [11]:
SEED = 42
PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = PROJECT_FOLDER + "train_images"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images"

IMAGE_SIZE = (224, 224)
VAL_SPLIT = 0.1

In [25]:
train_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='training',
    validation_split=VAL_SPLIT
)

val_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='validation',
    validation_split=VAL_SPLIT
)

# test_ds = image_dataset_from_directory(
#     directory=TEST_DATA_FOLDER,
#     labels=None,
#     batch_size=1,
#     seed=1337,
#     image_size=IMAGE_SIZE,
#     crop_to_aspect_ratio=True,
# )

Found 44703 files belonging to 3116 classes.
Using 40233 files for training.
Found 44703 files belonging to 3116 classes.
Using 4470 files for validation.


In [13]:
n_classes = len(train_ds.class_names)
n_classes

3116

In [14]:
def random_cutout(images_tensor, labels):
    ratios_x = rng.uniform(0.2, 0.6)*images_tensor.shape[1]
    ratios_y = rng.uniform(0.2, 0.6)*images_tensor.shape[2]

    xs_mask = tf.cast(ratios_x, tf.int32)
    xs_mask = xs_mask if xs_mask % 2 == 0 else xs_mask+1 # force number even

    ys_mask = tf.cast(ratios_y, tf.int32)
    ys_mask = ys_mask if ys_mask % 2 == 0 else ys_mask+1 # force number even

    return tfa.image.random_cutout(images_tensor, (xs_mask, ys_mask), constant_values = (255.0, 0.0, 0.0)), labels

train_ds = train_ds.map(random_cutout)
val_ds = val_ds.map(random_cutout)

# Models

In [15]:

def create_baseline_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB0(
            include_top=True,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    model = keras.Sequential([
        efficientnet_layer,
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1000, activation='selu'),
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1000, activation='selu'),
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
  


In [16]:
def create_finetuned_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB0(
            include_top= False,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    model = keras.Sequential([
        efficientnet_layer,
        keras.layers.GlobalAveragePooling2D(name = "avg_pool"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(n_classes, activation = 'softmax')
    ])
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [23]:
def create_unfreezed_efficientnet_model(model):
    for layer in model.layers[-25:]:
        if not isinstance(layer, keras.layers.BatchNormalization):
            layer.trainable = True
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [18]:
model = create_finetuned_efficientnet_model(n_classes)
model.summary()

16705208/16705208 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 avg_pool (GlobalAveragePool  (None, 1280)             0         
 ing2D)                                                          
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 3116)              3991596   
                                                         

In [20]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir = 'logs'
    ), 
    tf.keras.callbacks.ModelCheckpoint(
        filepath='ckp',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
]


history = model.fit(train_ds, epochs=15, callbacks=callbacks, validation_data=val_ds)

Epoch 1/15
1258/1258 [==============================] - 219s 173ms/step - loss: 2.4235 - accuracy: 0.6502 - val_loss: 15.1139 - val_accuracy: 0.2056
Epoch 2/15
1258/1258 [==============================] - 218s 172ms/step - loss: 2.4239 - accuracy: 0.6556 - val_loss: 15.0760 - val_accuracy: 0.2145
Epoch 3/15
1258/1258 [==============================] - 219s 173ms/step - loss: 2.4520 - accuracy: 0.6589 - val_loss: 15.8492 - val_accuracy: 0.2235
Epoch 4/15
1258/1258 [==============================] - 218s 172ms/step - loss: 2.4181 - accuracy: 0.6697 - val_loss: 16.4048 - val_accuracy: 0.2148
Epoch 5/15
1258/1258 [==============================] - 225s 178ms/step - loss: 2.4703 - accuracy: 0.6716 - val_loss: 16.8786 - val_accuracy: 0.2277
Epoch 6/15
1258/1258 [==============================] - 220s 173ms/step - loss: 2.4933 - accuracy: 0.6746 - val_loss: 17.1289 - val_accuracy: 0.2282
Epoch 7/15
1258/1258 [==============================] - 220s 174ms/step - loss: 2.5084 - accuracy: 0.6785 

In [22]:
model.save('../models/fine_tune_step1')

INFO:tensorflow:Assets written to: ../models/fine_tune_step1/assets


INFO:tensorflow:Assets written to: ../models/fine_tune_step1/assets


In [26]:
model = create_unfreezed_efficientnet_model(model)
history = model.fit(train_ds, epochs=25, callbacks=callbacks, validation_data=val_ds)

Epoch 1/25


2022-05-23 14:59:59.223302: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 323.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-05-23 14:59:59.223685: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 323.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-05-23 14:59:59.239771: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 509.59MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-05-23 14:59:59.239837: W tensorflow/core/common_runtime/bfc_al

   1/2515 [..............................] - ETA: 8:55:46 - loss: 19.2326 - accuracy: 0.1250

2022-05-23 15:00:11.835798: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.45MiB (rounded to 3612672)requested by op sequential/efficientnetb0/block6d_expand_bn/FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-05-23 15:00:11.835894: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-05-23 15:00:11.835908: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 466, Chunks in use: 466. 116.5KiB allocated for chunks. 116.5KiB in use in bin. 21.8KiB client-requested in use in bin.
2022-05-23 15:00:11.835915: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 240, Chunks in use: 236. 151.2KiB allocated for chunks. 148.5KiB in use in bin. 119.9KiB cl

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/efficientnetb0/block6d_expand_bn/FusedBatchNormV3' defined at (most recent call last):
    File "/home/pedro/miniconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/pedro/miniconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/pedro/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/pedro/miniconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/pedro/miniconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/pedro/miniconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/pedro/mini

568 next 1656
2022-05-23 15:00:11.850781: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 53ed98500 of size 8429568 next 1651
2022-05-23 15:00:11.850783: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 53f5a2500 of size 8429568 next 1662
2022-05-23 15:00:11.850785: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 53fdac500 of size 8429568 next 1666
2022-05-23 15:00:11.851094: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 5405b6500 of size 2107392 next 1664
2022-05-23 15:00:11.851099: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 5407b8d00 of size 2816 next 1668
2022-05-23 15:00:11.851101: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 5407b9800 of size 2816 next 1669
2022-05-23 15:00:11.851104: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 5407ba300 of size 2107392 next 1676
2022-05-23 15:00:11.851106: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse

In [ ]:
model.evaluate(test_ds)

1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [ ]:
model.save('empanadanet.h5')